### Practise

**Assembly Line Construction**
We have a simple, functional prototype application for monitoring an atmospheric processor, but it lacks modularity for future expansion. Our task is to refactor this prototype into a professional, scalable structure using an `application factory`.


**Preparation:**

We have this unorganized `processor_monitor.py` file with a recommended default configuration:
```python
# processor_monitor.py

from flask import Flask


app = Flask(__name__)
# --- Configuration ---
app.config['SECRET_KEY'] = 'a-secret-key-for-the-processor'
app.config['DEBUG'] = True


@app.route("/")
def index():
    return "<h1>Atmospheric Processor Monitor</h1><p>Status: Online</p>"

@app.route("/data")
def get_data():
    return {"pressure": "102.5 kPa", "oxygen_level": "20.9%"}

if __name__ == "__main__":
    app.run()
```

**Assignment:**
We will create a new "Assembly Line" project structure and move the code from `processor_monitor.py` into it:

1.  **"Assembly Line" structure:**
- processor_project/
  - wsgi.py
  - app/
    - _ _ init _ _.py
    - config.py
    - routes.py

2.  **`config.py`:** Move the configuration into the configuration file.
3.  **`routes.py`:** Move the routes and wrap them in a function.
4.  **Build the Application Factory** = in `app/__init__.py`: create a `create_app()` function that creates an instance of `app`, loads the configuration from `config.py`, and registers the routes from `routes.py`.
5.  **Create `wsgi.py`:** Write a startup script that imports and starts the application factory.

In [ ]:
# app/config.py

SECRET_KEY = 'a-secret-key-for-the-processor'
DEBUG = True

In [ ]:
# app/routes.py

def register_routes(app):
    @app.route("/")
    def index():
        return "<h1>Atmospheric Processor Monitor</h1><p>Status: Online</p>"

    @app.route("/data")
    def get_data():
        return {"pressure": "102.5 kPa", "oxygen_level": "20.9%"}

In [ ]:
# app/__init__.py

from flask import Flask
from . import config
from . import routes

def create_app():

    app = Flask(__name__)

    app.config.from_pyfile(config)

    routes.register_routes(app)

    return app

In [ ]:
# wsgi.py

from app import create_app

app = create_app()

if __name__ == "__main__":
    app.run()

### **Project (Homework): Upgrading the Core Control System**

**Mission:** Our current control system is functional, but it's a monolithic prototype. As the mission expands and tasks become more complex, it becomes cluttered and difficult to maintain. Our task is to perform the first phase of refactoring and rebuild the core of our application into a professional, scalable architecture using an application factory.


**Basic application structure:**
- my_project/
    - **wsgi.py**
    - **app/**
        - **_ _ init _ _.py**
        - routes.py
        - forms.py
        - database.py
        - **models/** 
            - **_ _ init _ _.py**
            - **crewmember.py**
        - data/
            - **_ _ init _ _.py**
            - asteroid.py
            - crew.py
        - templates/
        - static/
        - config/
    - **/instance**
        - **my_database.db**


Because our application has become a package with modules (and subpackages), the file with our **SQLite DB** will be created `**again** in the **instance** folder inside our project folder` - the application folder **/app** has become the **root** folder and the DB will be created "next to" it = in the project folder.


**1. Upgrading the Core Control System (Application Factory):** * Create a `wsgi.py` file, which will serve as the external launcher for our mission.
* Move all the logic for creating and configuring the application from the `my_app.py` file to a `create_app()` function in the `app/__init__.py` file = this is how we create the **application factory**.
* Modify `wsgi.py` to import and run the application from this factory.

**2. Standardizing Data Schemas:**
* Create a new **package** `app/models/`.
* Move the `CrewMember` model definition to a separate file `app/models/crew_member.py`.
* Create `app/models/__init__.py` (turning the folder into a package) and import the table model.
* Turn our `app/data` folder into a **package**.
* Create `__init__.py` for the `data` folder and import our older data about the asteroid and the crew.
* Modify the data imports in the files and in the application factory accordingly = we will use `relative path specification`.

**3. Systems Check:**
* After completing the modifications, test that the application still starts and all existing routes and functions work as they should.

In [ ]:
# app/config/configuration.py

class Config:
    """
    Set Flask config variables
    """
    # General Config
    DEBUG = True
    STATIC_FOLDER = 'static' 
    TEMPLATES_FOLDER = 'templates' 
    SECRET_KEY = 'my_secret_something' 

    # Database Config (set up DB connection) - for SQLite
    SQLALCHEMY_DATABASE_URI = 'sqlite:///my_database.db' 
    SQLALCHEMY_TRACK_MODIFICATIONS = False 

In [ ]:
# app/__init__.py

from .config.configuration import Config
from flask import Flask
from .database import db  
from .data import my_asteroid, my_crew 
from .models import CrewMember  
from .routes import register_routes


def app_factory():

    app = Flask(__name__)

    # CONFIGURACION
    app.config.from_object(Config) 

    # PLUGIN INIT - extensions for "app" object
    db.init_app(app) 

    # REGISTRATION (Routes)
    register_routes(app, db, my_asteroid) 
    
    # APP CONTEXT - where we need to know the structure of the app 
    with app.app_context():
        db.create_all() 
        if not CrewMember.query.first(): 
            for crew_member in my_crew:
                    new_crew_member = CrewMember(
                    id=crew_member["id"], 
                    name=crew_member["name"], 
                    specialization=crew_member["specialization"], 
                    status=crew_member["status"])
            db.session.add(new_crew_member)
            db.session.commit()

    return app

## /models

In [ ]:
# app/models/__init__.py

from .crewmember import CrewMember

In [ ]:
# app/models/crewmember.py

from ..database import db 


class CrewMember(db.Model):
    __tablename__ = "crew_members"
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    specialization = db.Column(db.String(100), nullable=False)
    status = db.Column(db.String(100), nullable=False)

    def __repr__(self):
        return f"<CrewMember {self.name}>"

## /data

In [ ]:
# app/data/__init__.py

from .asteroid import my_asteroid
from .crew import my_crew

In [ ]:
# data/asteroid

my_asteroid = {"name": "230 Athamantis",
               "diameter": "118±2 km",
               "mass": "(2.3±1.1)*10**18 kg",
               "density": "2.7±1.3 g/cm3",
               "speed": "19.3 km/s",
               "img": "static/img/Athamantis.jpeg"
}

In [ ]:
# data/crew

my_crew = [
        {
            "id": 1,
            "name": "Dave Fisher",
            "specialization": "AI Engineer",
            "status": "Active"
        },
        {
            "id": 2,
            "name": "Izael Alexander",
            "specialization": "Rocket Scientist",
            "status": "Active"
        },
        {
            "id": 3,
            "name": "Tethra Dyagran",
            "specialization": "Roboticist",
            "status": "Active"
        },
        {
            "id": 4,
            "name": "Mura Lan",
            "specialization": "Astronavigator",
            "status": "Active"
        }
    ]

In [ ]:
# app/routes.py

from flask import render_template, redirect, url_for, jsonify
from .forms import NewCrewMemberForm 
from .models import CrewMember


def register_routes(app, db, my_asteroid):
    @app.route("/")
    def homepage():
        return render_template("homepage.html")

    @app.route("/mission_briefing")
    def mission_briefing():
        return render_template("mission_briefing.html")

    @app.route("/target_asteroid")
    def target_asteroid():
        return render_template("target_asteroid.html", my_asteroid=my_asteroid)

    @app.route("/crew_members")
    def crew_members():
        my_crew = db.session.query(CrewMember).all()
        return render_template("crew_members.html", my_crew=my_crew)

    @app.route("/crew_members/<int:id>")
    def crew_members_details(id):
        crew_member = db.session.query(CrewMember).get(id)
        if crew_member:
            return render_template("crew_members_details.html", crew_member=crew_member)
        return "Crew member not found"
    
    @app.route("/onboarding", methods=["GET", "POST"])
    def onboarding():
        form = NewCrewMemberForm()
        if form.validate_on_submit():
            db.session.add(CrewMember(
                name=form.name.data,
                specialization=form.specialization.data,
                status="Pending.."
            ))
            db.session.commit()
            return redirect(url_for("crew_members"))
        return render_template("onboarding.html", form=form)
    

# -------------- API - ENDPOINTS --------------
    @app.route("/api/crew_members")
    def api_crew_members():
        all_crew_members = db.session.query(CrewMember).all()
        return jsonify(list({"id":member.id, "name":member.name, "specialization":member.specialization, "status":member.status} for member in all_crew_members)), 200
    
    @app.route("/api/target_asteroid")
    def api_target_asteroid():
        return jsonify(my_asteroid), 200

In [ ]:
# app/forms.py

from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField, validators

class NewCrewMemberForm(FlaskForm):
    name = StringField(label="Name", validators=[validators.DataRequired()])
    specialization = StringField(label="Specialization", validators=[validators.DataRequired()])
    
    submit = SubmitField(label="Submit new crew member")

In [ ]:
# app/database.py

from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

In [ ]:
# wsgi.py

from app import app_factory


app = app_factory()

if __name__ == "__main__":
    app.run(host='0.0.0.0')

## /templates

In [ ]:
<!-- header.html -->

<nav>
    <ul>
        <li><a href="{{ url_for('homepage') }}">Home</a></li> 
        <li><a href="{{ url_for('mission_briefing') }}">Our Mission</a></li>
        <li><a href="{{ url_for('target_asteroid') }}">The Asteroid</a></li>
        <li><a href="{{ url_for('crew_members') }}">Our Crew</a></li>
        <li><a href="{{ url_for('onboarding') }}">New Recruits Onboarding</a></li>
    </ul>
</nav>

In [ ]:
<!-- footer.html -->
 
<footer>
    <br>
    <p>Deep Space Exploration Corp.</p>
    <p>All rights reserved</p>
</footer>

In [ ]:
<!-- base.html -->
 
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block page_title %} Title {% endblock page_title %}</title> 
    
</head>

<body>
    {% include 'header.html' %} 
    {% block page_content %} Content {% endblock page_content %} 
    {% include 'footer.html' %} 
</body>

</html>

In [ ]:
<!-- homepage.html -->

{% extends "base.html" %} 

{% block page_title %} HomePage {% endblock page_title %}

{% block page_content %}
    <h1>Deep Space Exploration Corp.</h1>
    <p>Welcome to our homepage !</p>
    <p>Our goal is to mine asteroids from the solar system.</p>
{% endblock page_content %}

In [ ]:
<!-- mission_briefing.html -->

{% extends "base.html" %}

{% block page_title %} Mission Debriefing {% endblock page_title %}

{% block page_content %}
    <h1>Mission</h1>
    <p>This is our mission:</p>
    <p>get resources from chosen asteroids</p>
    <p>make it profitable</p>
    <p>expand world's economy</p>
{% endblock page_content %}

In [ ]:
<!-- target_asteroid.html -->

{% extends "base.html" %}

{% block page_title %} Target Asteroid {% endblock page_title %}

{% block page_content %}
    <h1>Target asteroid: {{ my_asteroid["name"] }}</h1>
    <p>Mean diameter: {{ my_asteroid["diameter"] }}</p>
    <p>Mass: {{ my_asteroid["mass"] }}</p>
    <p>Mean density: {{ my_asteroid["density"] }}</p>
    <p>Average orbital speed: {{ my_asteroid["speed"] }}</p>
    <img src="{{ my_asteroid['img'] }}" width="800px"/>  
{% endblock page_content %}

In [ ]:
<!-- crew_members_details.html -->

{% extends "base.html" %} 

{% block page_title %} {{ crew_member["name"] }} {% endblock page_title %}

{% block page_content %}
    <h1>Crew member {{ crew_member["id"] }}</h1>
    <p>Name: {{ crew_member["name"] }}</p>
    <p>Specialization: {{ crew_member["specialization"] }}</p>
    <p>Status: {{ crew_member["status"] }}</p>
{% endblock page_content %}

In [ ]:
<!-- crew_members.html -->

{% extends "base.html" %} 

{% block page_title %} Our Crew {% endblock page_title %}

{% block page_content %}
    {% for member in my_crew %}
        <p><a href="{{ url_for('crew_members_details', id=member['id']) }}">{{ member["name"] }}</a></p>
    {% endfor %}
{% endblock page_content %}

In [ ]:
<!-- onboarding.html -->

{% extends "base.html" %} 

{% block page_title %} Onboarding {% endblock page_title %}

{% block page_content %}
<form method = "POST" action = "{{ url_for('onboarding') }}">
    <p>New crew member onboarding form</p>
    {{ form.hidden_tag() }} 

    <p>{{ form.name.label }} {{ form.name(size=30) }}</p>
    <p>{{ form.specialization.label }} {{ form.specialization(size=30) }}</p>

    <p>{{ form.submit() }}</p>
</form>
{% endblock page_content %}

---
#### © Jiří Svoboda (George Freedom)
- Web: https://GeorgeFreedom.com
- LinkedIn: https://www.linkedin.com/in/georgefreedom/
- Book me: https://cal.com/georgefreedom